In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://localhost:4040
SparkContext available as 'sc' (version = 2.3.2, master = local[*], app id = local-1553468318695)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@48b2260b


In [2]:
%run "src/scala/register_tables.scala"

In [3]:
register_tables(spark,"spark-warehouse/")

===Prescription dataset===
registering patients
registering prescriptions
registering drugs
ok!
===Hospitalization dataset===
registering all
registering patients
registering prescriptions
registering drugs
ok!
===NPR Elders===
ok!
+--------+--------------------------+-----------+
|database|tableName                 |isTemporary|
+--------+--------------------------+-----------+
|        |elders                    |true       |
|        |elders_drugs              |true       |
|        |elders_patients           |true       |
|        |elders_prescriptions      |true       |
|        |npr_elders                |true       |
|        |prescription_drugs        |true       |
|        |prescription_patients     |true       |
|        |prescription_prescriptions|true       |
+--------+--------------------------+-----------+

done!


# NPR population: 176.817


In [4]:
val npr_raw = spark.read.option("header",true).csv("datasets/npr")
val npr_all_count = npr_raw.select("pasientlopenr").distinct.count
println(npr_all_count)

val npr_elders = spark.sql("select * from npr_elders")

val npr_elders_count = npr_elders.select("id").distinct.count
println(npr_elders_count)

println("Diff ",npr_all_count-npr_elders_count)

176817
172428
(Diff ,4389)


npr_raw: org.apache.spark.sql.DataFrame = [innmate: string, pasfylke: string ... 11 more fields]
npr_all_count: Long = 176817
npr_elders: org.apache.spark.sql.DataFrame = [id: string, num_hospitalizations: bigint ... 10 more fields]
npr_elders_count: Long = 172428


# Elders dataset
* Number of patients

In [5]:
val elders = spark.sql("select * from elders")
val total = elders.select("id").distinct.count
val hosp_ids = elders.where("diff_utleveringdato is not null")
    .select("id").distinct.count
val healthy_ids = elders.where("diff_utleveringdato is null")
    .select("id").distinct.count

elders: org.apache.spark.sql.DataFrame = [id: string, birthyear: int ... 21 more fields]
total: Long = 765431
hosp_ids: Long = 172428
healthy_ids: Long = 593003


In [6]:
elders.select("id","birthyear").distinct.sort(desc("birthyear"))
    .groupBy("birthyear").count.show(5)

+---------+-----+
|birthyear|count|
+---------+-----+
|     2012|    5|
|     2011|    4|
|     2010|    1|
|     2006|    1|
|     2004|    1|
+---------+-----+
only showing top 5 rows



### Number of hospitalized patients: 172.428
### Patients not hospitalized : 593.003
### Total: 765.431

# General dataset, total: 5.280.632

In [7]:
val general_patients = spark.sql("select * from prescription_patients")
    .count

general_patients: Long = 5280632


# Prescription counts
## Elders: 59.903.331
## General population: 370.164.691

In [8]:
val general_prescription_count = spark.sql("select * from prescription_prescriptions").select("drugcode").count
val elders_prescription_count = spark.sql("select * from elders_prescriptions").select("drugcode").count

println(general_prescription_count)
println(elders_prescription_count)

370164691
59903331


general_prescription_count: Long = 370164691
elders_prescription_count: Long = 59903331


In [19]:
spark.sql("select * from elders_prescriptions")
    .select("timestamp")
    .where(col("timestamp") < lit("2010.01.01"))
    .count

res14: Long = 0


In [20]:
spark.sql("select * from elders_prescriptions")
    .select("timestamp")
    .where(col("timestamp") < lit("2011.01.01"))
    .count

res15: Long = 0


In [21]:
spark.sql("select * from elders_prescriptions")
    .select("timestamp")
    .where(col("timestamp") < lit("2012.01.01"))
    .count

res16: Long = 12594935


In [22]:
spark.sql("select * from elders_prescriptions")
    .select("timestamp")
    .where(col("timestamp") < lit("2013.01.01"))
    .count

res17: Long = 25967931


In [23]:
spark.sql("select * from elders_prescriptions")
    .select("timestamp")
    .where(col("timestamp") > lit("2013.01.01"))
    .count

res18: Long = 13647042


In [24]:
spark.sql("select * from elders_prescriptions")
    .select("timestamp")
    .where(col("timestamp") > lit("2014.01.01"))
    .count

res19: Long = 0


In [25]:
spark.sql("select * from elders_prescriptions")
    .select("timestamp")
    .where(col("timestamp") > lit("2015.01.01"))
    .count

res20: Long = 0
